# Generate experiment csv

In [1]:
import json
from copy import deepcopy
import pandas as pd

In [ ]:
# CONFIG_PATH = "experiment_values.json"

# with open(CONFIG_PATH) as f:
#     config = json.load(f)

# defaults = config["defaults"]
# grids = config["grids"]
# experiments = config["experiments"]

# for agent, sweep in experiments.items():
# 	default_params = defaults[agent]

# 	for grid in grids:
# 		for param_name, values in sweep.items():
# 			for val in values:
# 				params = deepcopy(default_params)
# 				params[param_name] = val

# 				run_config = {
# 					"agent": agent,
# 					"grid": grid,
# 					"param_changed": param_name,
# 					"param_value": val,
# 					"params": params
# 				}

# 				cli_args = []
# 				for k, v in run_config["params"].items():
# 					cli_args += [f"--{k}", str(v)]

In [ ]:
# cli_args

['--iter', '5000']

In [4]:
CONFIG_PATH = "experiment_values.json"

with open(CONFIG_PATH) as f:
    config = json.load(f)

defaults = config["defaults"]
grids = config["grids"]
experiments = config["experiments"]

all_param_names = set()
for agent_params in defaults.values():
    all_param_names.update(agent_params.keys())

rows = []

for agent, sweep in experiments.items():
    default_params = defaults[agent]

    for grid in grids:
        for param_name, values in sweep.items():
            for val in values:
                params = deepcopy(default_params)
                params[param_name] = val

                row = {
                    "agent": agent,
                    "grid": grid,
                    "param_changed": param_name,
                    "param_value": val
                }

                for pname in sorted(all_param_names):
                    row[pname] = params.get(pname, float('nan'))

                rows.append(row)

df = pd.DataFrame(rows)

df.to_csv("experiment_table.csv", index=False)

In [ ]:
df = df.loc[df["agent"] != "RandomAgent"]
df.to_csv("experimental_table.csv", index = False)

# Training - Copy + Minor update to train.py code. Train.py wasn't touched

In [6]:
import json
import numpy as np
import pandas as pd
from copy import deepcopy
from argparse import Namespace
from tqdm import trange
from pathlib import Path
import importlib
import inspect
from inspect import Parameter

from world.reward_functions import custom_reward_function
from world import Environment
from agents import BaseAgent

pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
def load_agent(agent_name: str, env: Environment, config: dict) -> tuple[BaseAgent, str]:
    agent_info = config[agent_name]
    module = importlib.import_module(agent_info["module"])
    AgentClass = getattr(module, agent_info["class"])
    init_args = agent_info.get("init_args", {})

    sig = inspect.signature(AgentClass.__init__)
    if 'env' in sig.parameters:
        agent = AgentClass(env=env, **init_args)
    else:
        agent = AgentClass(**init_args)

    return agent, agent_info["train_mode"]

def update_agent(agent: BaseAgent, args: Namespace,
                 state: tuple[int, int],
                 next_state: tuple[int, int],
                 reward: float,
                 actual_action: int) -> None:
    update_params = inspect.signature(agent.update).parameters
    update_param_names = list(update_params)

    if {"state", "next_state"}.issubset(update_param_names):
        agent.update(state=state, next_state=next_state, reward=reward, action=actual_action)
    elif {"next_state", "reward", "action"}.issubset(update_param_names):
        agent.update(next_state=next_state, reward=reward, action=actual_action)
    elif {"state", "reward", "action"}.issubset(update_param_names):
        agent.update(state=state, reward=reward, action=actual_action)
    elif all(p.kind in {Parameter.VAR_POSITIONAL, Parameter.VAR_KEYWORD} for p in update_params.values()):
        agent.update()
    else:
        raise ValueError(f"Unsupported update() signature: {update_param_names}")

def train_agent(args: Namespace, config: dict):
	start_pos = tuple(args.agent_start_pos)
	for grid_path in args.GRID:
		env = Environment(
			Path(grid_path), args.no_gui, sigma=args.sigma, agent_start_pos=start_pos,
			reward_fn=custom_reward_function, target_fps=args.fps, random_seed=args.random_seed
		)
		env.reset()
		agent, mode = load_agent(args.agent, env, config)

		if mode == "episodic":
			delta = 1e-6
			for ep in trange(args.episodes, desc=f"Training {args.agent}"):
				prev_q = {s: np.copy(q) for s, q in agent.q_table.items()}
				state = env.reset()
				for _ in range(args.iter):
					action = agent.take_action(state)
					next_state, reward, terminated, info = env.step(action)
					if terminated: break
					agent.update(state, next_state, reward, info["actual_action"])
					state = next_state
				common = set(agent.q_table.keys()) & set(prev_q.keys())
				max_diff = max(
					np.max(np.abs(agent.q_table[s] - prev_q[s]))
					for s in common
				) if common else 10
				if max_diff < delta:
					break
			agent.eval_mode()

		elif mode == "iterative":
			state = env.reset()
			for _ in trange(args.iter, desc=f"Training {args.agent}"):
				action = agent.take_action(state)
				next_state, reward, terminated, info = env.step(action)
				update_agent(agent, args, state, next_state, reward, info["actual_action"])
				state = next_state
				if terminated: break

		# Environment.evaluate_agent(
		#     grid_path, agent, args.iter, args.sigma, agent_start_pos=start_pos,
		#     reward_fn=custom_reward_function, random_seed=args.random_seed
		# )
		Environment.evaluate_agent(
			Path(grid_path), agent, args.iter, args.sigma, agent_start_pos=start_pos,
			reward_fn=custom_reward_function, random_seed=args.random_seed
		)


### Can just regen df instead of reading

In [9]:
df = pd.read_csv("/Users/aniket/TU_Eindhoven/2_Study/Q4_S2AMC15_Data_Intelligence_Challenge/4_Code/2AMC15_Intelligence_Challenge/experimental_table.csv")  # or assign df directly
with open("agent_config.json") as f:
    base_config = json.load(f)

TRAIN_ARGS = {"episodes", "iter"}

In [10]:
for i, row in df.iterrows():
    agent = row["agent"]
    grid = row["grid"]

    print(f"Row {i+1}: {agent} on {grid} | {row['param_changed']} = {row['param_value']}")

    init_args = {}
    train_args = {}

    for col, val in row.items():
        if pd.isna(val): continue
        if col in {"agent", "grid", "param_changed", "param_value"}:
            continue
        if col in TRAIN_ARGS:
            train_args[col] = int(val)
        else:
            init_args[col] = float(val) if isinstance(val, float) else val

    config = deepcopy(base_config)
    config[agent]["init_args"] = init_args
    args = Namespace(
        GRID=[f"grid_configs/{grid}.npy"],
        agent=agent,
        no_gui=True,
        sigma=0.1,
        fps=5,
        episodes=train_args.get("episodes", 2000),
        iter=train_args.get("iter", 2000),
        random_seed=42,
        agent_start_pos=[1, 1],
    )

    train_agent(args, config)


Row 1: QLearningAgent on A1_grid | gamma = 0.6


Evaluating agent:   1%|          | 15/2000 [00:00<00:00, 122164.19it/s]


Evaluation complete. Results:
cumulative_reward: 35
total_steps: 16
total_agent_moves: 16
total_failed_moves: 0
total_targets_reached: 1
targets_remaining: 0
Row 2: QLearningAgent on A1_grid | gamma = 0.95


Evaluating agent:   1%|          | 15/2000 [00:00<00:00, 159681.62it/s]


Evaluation complete. Results:
cumulative_reward: 35
total_steps: 16
total_agent_moves: 16
total_failed_moves: 0
total_targets_reached: 1
targets_remaining: 0
Row 3: QLearningAgent on A1_grid | alpha = 0.3


...

Evaluation complete. Results:
cumulative_reward: 30
total_steps: 21
total_agent_moves: 21
total_failed_moves: 0
total_targets_reached: 1
targets_remaining: 0
Row 47: ValueIterationAgent on large_grid | theta = 1e-05


Evaluating agent:   1%|          | 20/2000 [00:00<00:00, 290263.25it/s]


Evaluation complete. Results:
cumulative_reward: 30
total_steps: 21
total_agent_moves: 21
total_failed_moves: 0
total_targets_reached: 1
targets_remaining: 0
Row 48: ValueIterationAgent on large_grid | iter = 5000.0


Evaluating agent:   0%|          | 20/5000 [00:00<00:00, 274137.52it/s]

Evaluation complete. Results:
cumulative_reward: 30
total_steps: 21
total_agent_moves: 21
total_failed_moves: 0
total_targets_reached: 1
targets_remaining: 0
